 [Iris数据集](https://en.wikipedia.org/wiki/Iris_flower_data_set)是常用的分类实验数据集，由Fisher,1936收集整理。Iris也称鸢尾花卉数据集，是一类多重变量分析的数据集。数据集包含150个数据集，分为3类，每类50个数据，每个数据包含4个属性。可通过花萼长度，花萼宽度，花瓣长度，花瓣宽度4个属性预测鸢尾花卉属于（Setosa，Versicolour，Virginica）三个种类中的哪一类。   
iris以鸢尾花的特征作为数据来源，常用在分类操作中。该数据集由3种不同类型的鸢尾花的50个样本数据构成。其中的一个种类与另外两个种类是线性可分离的，后两个种类是非线性可分离的。   

该数据集包含了4个属性：  
Sepal.Length（花萼长度），单位是cm;  
Sepal.Width（花萼宽度），单位是cm;  
Petal.Length（花瓣长度），单位是cm;  
Petal.Width（花瓣宽度），单位是cm;  
种类：  
Iris Setosa（1.山鸢尾）、Iris Versicolour（2.杂色鸢尾），以及Iris Virginica（3.维吉尼亚鸢尾）。

# 参考资料
[python from __future__ import division](https://blog.csdn.net/feixingfei/article/details/7081446)

In [35]:
from __future__ import division
import math
import random
import pandas as pd

In [36]:
flowerLables = {0:'Iris-setosa', 1:'Iris-versicolor', 2:'Iris-virginica'}
random.seed(0)
  
# 生成区间[a, b)内的随机数
def rand(a, b):
    return (b - a) * random.random() + a  # random() 方法返回随机生成的一个实数，它在[0,1)范围内
  
# 生成大小 I*J 的矩阵，默认零矩阵
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill] * J)
    return m
  
# 函数 sigmoid
def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))
  
# 函数 sigmoid 的导数
def dsigmoid(x):
    return x * (1 - x)

In [46]:
class BPNN:
    """ 三层反向传播神经网络 """
 
    def __init__(self, ni, nh, no):
        # 输入层、隐藏层、输出层的节点（数）
        self.ni = ni + 1  # 增加一个偏差节点
        self.nh = nh + 1
        self.no = no
 
        # 激活神经网络的所有节点（向量）
        self.ai = [1.0] * self.ni
        self.ah = [1.0] * self.nh
        self.ao = [1.0] * self.no
 
        # 建立权重（矩阵）
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # 设为随机值
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2, 2)
 
    def update(self, inputs):
        if len(inputs) != self.ni - 1:
            raise ValueError('与输入层节点数不符！')
 
        # 激活输入层
        for i in range(self.ni - 1):
            self.ai[i] = inputs[i]
 
        # 激活隐藏层
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)
 
        # 激活输出层
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)
 
        return self.ao[:]  
        # 发现用b=a[:], 再给b赋值, 不会影响a; 直接给a[:]赋值却会影响a  
        # b=a[:] 先是 a[:] 生成了一个新的对象，然后 b 指向新对象，a 没变。
 
    def backPropagate(self, targets, lr):
        """ 反向传播 """
 
        # 计算输出层的误差
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k] - self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error 
   
        # 计算隐藏层的误差
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error
 
        # 更新输出层权重
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] = self.wo[j][k] + lr * change
 
        # 更新输入层权重
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j] * self.ai[i]
                self.wi[i][j] = self.wi[i][j] + lr * change
 
        # 计算误差
        error = 0.0
        error += 0.5 * (targets[k] - self.ao[k]) ** 2
        return error
 
    def weights(self):
        print('输入层权重:')
        for i in range(self.ni):
            print(self.wi[i])
        print('输出层权重:')
        for j in range(self.nh):
            print(self.wo[j])
 
    def train(self, patterns, iterations=1000, lr=0.1):
        # lr: 学习速率(learning rate)
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, lr)
            if i % 100 == 0:
                print('error: %-.9f%%' % error)
    
    def test(self, patterns):
        count = 0
        for p in patterns:
            target = flowerLables[(p[1].index(1))]
            result = self.update(p[0])
            index = result.index(max(result))
            print(p[0], ':', target, '->', flowerLables[index])
            count += (target == flowerLables[index])
        accuracy = float(count / len(patterns))
        print('accuracy: %-.9f' % accuracy)

In [47]:
def iris():
    data = []
    # 读取数据
    raw = pd.read_csv('iris.txt')  # 读取为DataFrame格式
    raw_data = raw.values
    raw_feature = raw_data[0:, 0:4]
    for i in range(len(raw_feature)):
        ele = []
        ele.append(list(raw_feature[i]))
        if raw_data[i][4] == 'Iris-setosa':
            ele.append([1, 0, 0])
        elif raw_data[i][4] == 'Iris-versicolor':
            ele.append([0, 1, 0])
        else:
            ele.append([0, 0, 1])
        data.append(ele)
#     print(data)  # 观察这块的输出结果
    # 随机排列数据
    random.shuffle(data)
    training = data[0:100]
    test = data[101:]
    nn = BPNN(4, 7, 3)
    nn.train(training, iterations=10000)
    nn.test(test)

In [48]:
 if __name__ == '__main__':
    iris()

error: 12.357477256%
error: 1.823539310%
error: 1.426465911%
error: 1.329722320%
error: 1.309893890%
error: 1.304054262%
error: 1.273563779%
error: 1.215616854%
error: 1.149312496%
error: 1.107724232%
error: 1.087725592%
error: 1.072673716%
error: 1.060761604%
error: 1.050948109%
error: 1.042257415%
error: 1.035462930%
error: 1.032060474%
error: 1.032894798%
error: 1.038540591%
error: 1.049716714%
error: 1.065111238%
error: 1.069363023%
error: 0.855694255%
error: 0.839561643%
error: 0.825896145%
error: 0.813651318%
error: 0.802477625%
error: 0.792355145%
error: 0.783480986%
error: 0.775327938%
error: 0.763909728%
error: 0.749069705%
error: 0.732690947%
error: 0.714913888%
error: 0.695553267%
error: 0.674360213%
error: 0.651097934%
error: 0.625840882%
error: 0.599822096%
error: 0.576073475%
error: 0.557297923%
error: 0.543555954%
error: 0.533502265%
error: 0.525880451%
error: 0.519847116%
error: 0.514877200%
error: 0.510643534%
error: 0.506937049%
error: 0.503619990%
error: 0.500599013%